# Statistical birds eye view of the contents in an AiiDAdb

This is the first of two deliverable for the SiSc-Lab2020 project.

Authors = Miao Wang(a - e), Zhipeng Tan(f - i)

Supervisors: Jens Bröder, Dr. Daniel Wortmann, Johannes Wasmer, Prof. Dr. Stefan Blügel.

In [ ]:
# Instructions by supervisors

## Jens
a= """
You have to implement this notebook.

In the end only text (markdown) cells and output results of code cells should be seen if one hides the code cells (hide_code extension).

That can easily exported into a nice pdf file (google it, probably will find sth with `nbconvert`).

Also the amount of python code in this notebook should be minimal.

Rather, export the functions you use to python file(s) and import them here (hide complexity).

Optional dump query results in a file, from which results will be reread for speed, i.e cache results.
"""

## Johannes
a = '''
After talking with Jens about it, here are some additional thoughts on the code structure and implementation, for both deliverables.

The **primary** goal is of course that the code should work, produce nice output (and helpful error messages), obviously.

The **secondary** goal is speed. How long do you expect your code to run on a dataset of a given size? Are there multiple paths to a goal, but with differing performance?

You can break the runtime down into several steps: data acquisition, data transformation (or preprocessing), data analysis, data visualization. In this project, we will rename/replace these steps to: **querying, de-/serialization, analysis=visualization**.

**Querying the database.** Performance considerations:
- Performance measurement: use the magics `%time` and `%timeit`.
- Query evaluations: queries (in general) use 'lazy evaluation'.
  - *Query building* methods build the query but do not execute it. These are chainable methods like `append()`, `get_outgoing()`, etc.
  - *Query execution* methods send the query to the database to be evaluated. There are two kinds:
    - non-iterator methods: e.g. `all()`, `first()`, etc. These return a result `list`: all items are loaded into memory.
    - iterator methods: e.g. `iterall()`, `iterdict()`. These return a result `Generator`: only one item at a time is loaded into memory.
    
**De-/serialization**, i.e. writing and reading it to/from a file. *Keep in mind: if you come to the conclusion this is unnecessary, then justify it!* Considerations:
- Necessity: we assume 'yes'. So you need serialization/deserialization routine(s).
- Code design: we recommend to write a serializer that moves *all* data needed from aiida to file (perform query & serialization). Then the visualization methods are decoupled from aiida and load data from that file. Advantages: a) only needs to be called when data in database changed, b) similar queries for different visualizations can be performed only once. One design option is this:
  ```python
  serialize = sisclab.Serializer(profile)
  serialize.to_file(filepath)
  visualize = sisclab.Visualizer(filepath)
  visualize.histogram(cumulative=True, plot_options)
  # plots histogram
  ```
- Serialization format: there are two practical options (maybe more):
  - `dict`: tree-like. JSON format. One `dict` per file. choose key-value (nested?) based on use-case. in general, `uuid` is a good key.
  - `pandas.Dataframe`: could be preferrable in some cases.
- Serialization location:
  - one file or several files?
  - we recommend to de/serialize from/to `sisclab/data/` folder. It is included in the project's `.gitignore` file, so nothing in it gets committed to/from git (git is for code, not for data; the code generates the data).
- Transformation:
  - if needed, decide where to put needed data transformations (before serialization or after deserialization) to minimize them.
- Deserialization: 
  - a class (as above) might help to define the deserialization format only once for all visualization methods.
  

**Visualization**:
- Prefer `bokeh` to `matplotlib` or other libs wherever possible, unless you have a good justification.
- In `D1`, static plots are okay, interactive plots are a bonus.
- Lists results (when plot is overkill) will look nicer in a notebook if they are a `pandas.Series` or `pandas.Dataframes`.
- Think about function signatures. Can you generalize them to make a nice interface? For example, a signature for SubtaskD1.c might look like this:
  ```python
  def node_type_summary(user_list : list = [], node_basetype : Node = Data,
                        chart_type : bokeh.chart_type = bokeh.pie_chart, plot : bool = True):
    """
    :param user_list: list of users. empty list = all users = default.
    :param node_basetype: subdivides chart into subtypes. Valid base type examples: ProcessNode, CalculationNode, WorkflowNode, Data, ArrayData.
    :param chart_type: bokeh visualization type. pie chart = default.
    :param plot: True: show plot, don't return data. False: don't plot, return data.
    :return: stats: a dictionary {node_subtype : node_count}, insertion-order sorted in descending order.
    :rtype: dict.
    """
  ```
'''

In [ ]:
# Imports

In [35]:
# magics:
# # autoreload imports. 
# # intent: if i change sth in import, i don't have to restart kernel. enable only for development.
%load_ext autoreload
%autoreload 2
# # choose matplotlib backend. backend 'notebook' allows interactive plots if your env allows it.
%matplotlib notebook


In [4]:
# python imports:
from collections import Counter
import time
#from pprint import pprint

#%pylab inline
#figuresize=(18, 4)
from collections import Counter
from math import pi
import pandas as pd
from pandas import DataFrame
import numpy as np
from bokeh.io import output_file,output_notebook, show
from bokeh.layouts import column
from bokeh.palettes import Category20,Category20c
from bokeh.plotting import figure,ColumnDataSource
from bokeh.transform import cumsum
from bokeh.models import Legend,LegendItem,HoverTool

# aiida imports:
from aiida import load_profile
profile = load_profile()

# ggf add futher imports
from aiida.orm import QueryBuilder as QB
from aiida.orm import ProcessNode,Node, User, CalcJobNode, Computer, Code,Dict
from aiida.plugins import DataFactory

from aiida.common.constants import elements as PeriodicTableElements

# project imports:
import helpers
# from aiida_jutools.sisc_lab import helpers
# from aiida_jutools.sisc_lab import HelpersPackage
# equivalent ('.' is the sisc_lab directory):
# from . import helpers
# alternative:
# from .helpers import print_bold
# from .helpers import * ('*' import everything; use of '*' is considered bad style)

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
# (example:)
helpers.print_bold(f"This notebook/dashboard will visualize the contents from the database of profile {profile.name}")

This notebook/dashboard will visualize the contents from the database of profile generic


# Database overview:

In [ ]:
# SubtaskD1.a: Node information
#Task:

In [ ]:
# query for all nodes
print('Information on nodes in the DB: \n')
now = time.strftime("%c")
print('last executed on {}'.format(now))
q = QB()
q.append(Node, project=['id', 'ctime', 'mtime', 'node_type'], tag='node')
q.append(User, with_node='node', project='email')
# TODO: execute query here
t = time.time()
res = q.all()
elapsed = time.time() - t
totalnodes = len(res)
print("Total number of nodes in the database: {} (retrieved in {} s.)".format(totalnodes, elapsed))

## User information:

In [ ]:
# SubtaskD1.b: Users
a = '''
Task: print out a list of Users and how many nodes belong to them

for example

```
Users:
- j.broeder@fz-juelich.de created 182 nodes
- tests@aiida.mail created 104 nodes
```
'''

In [ ]:
print("Users:")
helpers.print_Count('user',res)

## Node types distribution:

In [ ]:
# SubtaskD1.c: Node types
a = '''
Task: plot node information in two pie chart plots

One showing what data nodes there (with their lowest class names(node_type)) I.e Dict, K-pointsData, CifData, FleurinpData...

And one chart showning the process nodes, (with their lowest class names(process_type) i.e CalcjobNodes: FleurCalcjob, FleurinputgenCalcjob, ...

WorkChain nodes: FleurSCFWorkchain, FleurBandDosWorkchain, ..., calcfunctions, and workfunction nodes are fine to not show the lowest class names
'''

In [ ]:
#node types
print("Node types:")
helpers.print_Count('types',res)

In [ ]:
#split data nodes and process nodes
output_notebook()
types = Counter([r[3] for r in res])
x = helpers.get_data_node_count(types,'data') 
p = helpers.draw_pie_chart(x,'Data Nodes')

x1 = helpers.get__process_node_count(types,'process')
p1 = helpers.draw_pie_chart(x1,'Process Nodes')

show(column(p,p1))


In [ ]:
output_notebook()
p = helpers.draw_pie_chart(Counter(helpers.get_dict_link_types()),'Dict Link Types')
show(p)

## Database time evolution:

In [ ]:
# SubtaskD1.d: Histogram
# Task: Cumulative Histogram/ or line plot by ctime & mtime of all nodes over time

In [ ]:
# line plot by ctime & mtime
users = Counter([r[4] for r in res])
output_notebook()
helpers.draw_line_plot(users,res)

## Codes:

In [ ]:
# SubtaskD1.e: Codes
#Task: List Code names, sorted by by how many calcjobs where run with each

In [ ]:
codes = Code.objects.all()
result = {code.full_label: len(code.get_outgoing(node_class=CalcJobNode).all_nodes()) for code in codes}
#result_df=pd.Series(result).sort_values(ascending=False)
result_df=pd.DataFrame({'code@computer':result.keys(),'CalaJobcount':result.values()}).sort_values(by='CalaJobcount',ascending=False).reset_index(drop=True)
result_df

## Groups:

In [ ]:
# SubtaskD1.f: Groups
#Task: List all group names with how many nodes they contain (verdi group list -C) (exclude import and export groups)

In [1]:
#load_profile()
#!verdi group list --all
import pandas as pd

import HelperPackage.DataProcessing.DataVisu as DV
import HelperPackage.Serialization.Serialization as SR
from aiida.orm import QueryBuilder
from aiida.orm import load_node, Node, Group, Computer, User, CalcJobNode, Code, StructureData

## this part need to be serialized and de-serialized
qb = QueryBuilder()
qb.append(Group)
group = qb.all()

#data = GroupDataHelper(group)
#data.ListGroup(exclude=['export','import'])

/home/max/Desktop/sisclab new/aiida-jutools/aiida_jutools/sisc_lab/HelperPackage loaded successfully
DataProcess Package Initialization successfully
Aiida file loaded successfully


In [3]:
### add more columns for this and do also for other nodes
serializer = SR.Serializer(group)
serializer.to_file('./output/group.json',Node_type='Group')

0

In [4]:
x = pd.read_json('./output/group.json')
x

,User,Group_Name,Node,type_string
0,gianluca.prandini@epfl.ch,SSSP_1.1_efficiency,85,core.upf
1,aiida@localhost,20201209-155346,85,core.import
2,gianluca.prandini@epfl.ch,SSSP_1.1_precision,85,core.upf
3,aiida@localhost,20201209-155429,85,core.import
4,aiida@localhost,sg15-oncv-1.1,69,core.upf
5,j.broeder@fz-juelich.de,codes,18,core
6,j.broeder@fz-juelich.de,fusion_materials,79,core
7,j.broeder@fz-juelich.de,stable_binary_metals_from_materialsproject,5058,core
8,j.broeder@fz-juelich.de,fleur_initial_cls_finished,7,core
9,j.broeder@fz-juelich.de,delta_parameters_gutstav_soc,71,core


In [6]:
data = DV.GroupDataHelper(x)
data.ListGroup(exclude=['export','import'])

Group names:                                        sizes:
SSSP_1.1_efficiency                               |   85
SSSP_1.1_precision                                |   85
sg15-oncv-1.1                                     |   69
codes                                             |   18
fusion_materials                                  |   79
stable_binary_metals_from_materialsproject        | 5058
fleur_initial_cls_finished                        |    7
delta_parameters_gutstav_soc                      |   71
fusion_strucs_refinded                            |   23


## Structure Analysis:

In [ ]:
# SubtaskD1.g: Structures
a = '''
Task: Further analyze what structures are in the DB

Number of structureData node versus how many atoms they contain. 

here interactive with bokeh hover tool showing the structure formula and uuid

Number of StructureData nodes versus elements bokeh bar chart, since there are over 
100 elements in the periodic table you can split it over several plots, or just use the charge number as in 
'example/element_content.png' but then make it interactive that once one hovers 
with the mouse over a bar it tells you what element it is and how many structures there are containing this element-
'''

import numpy as np
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource,HoverTool
from bokeh.plotting import figure
from bokeh.io import output_notebook
from bokeh.palettes import inferno
from HelperPackage.DataProcessing.DataVisu import StrucFormulaHelper
import pandas as pd
    
def AnalyseStructureElements():
    '''
    This function count the Elements and number of StructureNode, the Structure Node is the 
    '''
    #### return the pd.DataFrame including elements and number of each element

    StrucList = []
    qb = QueryBuilder()
    qb.append(StructureData)
    print('number of StructureData Nodes:',qb.count())
    qb.count()

    for struc, in qb.all()[:]:
        form = struc.get_formula()
        struct = StrucFormulaHelper(form)
        StrucList = StrucList+ [struct.FormAnalyse()]
    return pd.DataFrame(StrucList).fillna(0)

def ShowElements(Data):
    #### visualize the Elements and number of them
    output_file("ShowingElements.html")
    #data = NumStructureNode()
    data = Data
    elements = list(data.columns)
    counts = list(data.astype(bool).sum(axis=0))
    # zip sort
    counts,elements = zip(*sorted(zip(counts,elements)))
    
    #print(counts)
    #print(elements)
    
    source = ColumnDataSource(data=dict(elements=elements, counts=counts,color=inferno(len(elements))))
    
    TOOLTIPS = [
    ("element", "@elements"),
    ("(x,y)", "($x, $y)"),
    ("Number of Structures containing this element", "@counts"),
    ]

    p = figure( y_range=elements,x_range=(0,np.max(counts)), plot_width=800, plot_height=800, title="Elements Counts",tools = [HoverTool(mode='hline')], tooltips=TOOLTIPS)
    #print('step figure done')
    p.hbar(y="elements", right="counts", height=0.5, left=0, color='color',  source=source)
    #print('step hbar done')
    
    output_notebook()
    p.xgrid.grid_line_color = None
    #p.legend = False
    show(p)
    
    
def ShowFormula(Data):
    ## Show the formula and id
    

    output_file("ShowingFormula.html")
    data = Data
    elements = list(data.keys())
    counts = list(len(data[key])/2 for key in data.keys())
    formulas = list(data[key][:10] for key in data.keys())

    length = len(elements)
    source = ColumnDataSource(data=dict(elements=elements, counts=counts,formulas=formulas,color=inferno(length)))
    
    TOOLTIPS = [
    ("Number of Atoms", "@elements"),
    ("(x,y)", "($x, $y)"),
    ("Number of Nodes", "@counts"),
    ("Id and formula(first 5 nodes of all)", "@formulas"),
    ]

    p = figure(x_range=(0,np.max(counts)+20),y_range=(0,np.max(elements)), plot_width=800, plot_height=800, title="Atoms Count",tools = [HoverTool(mode='hline')], tooltips=TOOLTIPS)
    #print('step figure done')
    p.hbar(y="elements", right="counts", height=0.5, left=0, color='color',  source=source)
    #print('step hbar done')
    
    output_notebook()
    p.xgrid.grid_line_color = None
    #p.legend = False
    show(p)
    
##############################################    
def GetFormulaDict():
    ## may be not necessary, return the formulas and nodes containing this formula
    import numpy as np
    from aiida.orm import QueryBuilder,StructureData
    qb = QueryBuilder()
    qb.append(StructureData)
    Mydict = qb.all()
    Mydict = np.ravel(Mydict)

    Newdict = {}
    for dict in Mydict:
        if dict.get_formula in Newdict.keys():
            Newdict[dict.get_formula()].append(dict.uuid)
        else:
            Newdict[dict.get_formula()] = [dict.uuid]
    return Newdict
############################################


In [7]:
################### serialization
import HelperPackage.Serialization.Serialization as SR
qb = QueryBuilder()
qb.append(StructureData)
StructDatas = qb.all()

    
import json
#print(dic.keys())

serializer = SR.Serializer(StructDatas)
serializer.to_file('./output/Num_structure.json' ,Node_type='StructureFormula')


0

In [8]:
########### deserialize and show    

import HelperPackage.Serialization.Serialization as SR

filepath = './output/Num_structure.json'
Newdata = SR.deserialize_from_file(filepath,Node_type = 'StructureFormula')

DV.ShowFormula(Newdata)

Loading BokehJS ...

In [9]:
from HelperPackage.DataProcessing.DataVisu import AnalyseStructureElements,ShowElements

from aiida.orm import QueryBuilder
from aiida.orm import load_node, Node, Group, Computer, User, CalcJobNode, Code, StructureData
import HelperPackage.Serialization.Serialization as SR

qb = QueryBuilder()
qb.append(StructureData)
StructDatas = qb.all()

In [10]:
serializer = SR.Serializer(StructDatas)
filepath = './output/Struct_Element.json'
serializer.to_file(filepath,'StructureElement')

0

In [13]:
x = SR.deserialize_from_file(filepath,'StructureElement')

In [14]:
ShowElements(x)

Loading BokehJS ...

# initialize the Structure data
from aiida.orm import QueryBuilder
from aiida.orm import load_node, Node, Group, Computer, User, CalcJobNode, Code, StructureData
from HelperPackage import DataProcessing
from DataProcessing.DataVisu import StrucDataHelper

qb = QueryBuilder()
qb.append(StructureData)

structures = qb.all()

for structure in structures[:]:
    formula = structure[0].get_formula()
    struct = StrucDataHelper(formula)
    structure[0].get_composition()
    #print(struct.FormAnalyse())

## Processes:

In [ ]:
# SubtaskD1.h: Calculations
a = '''
Task: more detail analysis of Calculations

`print('\n\nMore detailed analysis of Calculations \n')`

List, stacked Histogram of Calculations types and the state it ended up finished, failed, exit codes, exit messages

more detail analysis of WorkChains

`print('\n\nMore detailed analysis of WorkChains \n')`

List,  stacked Histogram for each Workchain type and the state it ended up in finished, failed, exit codes, exit messages
'''

def GetCalNode():
    exit_state = []
    exit_message = []
    index = []
    exit_state_digit = []
    for node, in CalcNode:
        #print(str(node.process_state))
        #print(node.pk)
        #print(str(node.exit_message))
        exit_state = exit_state + [str(node.process_state)]
        exit_message = exit_message + [str(node.exit_message)]
        index = index + [node.pk]
        if node.is_finished_ok:
            exit_state_digit = exit_state_digit + [1]
        else:
            exit_state_digit = exit_state_digit + [0]
    return exit_state,exit_message,index,exit_state_digit

def ShowCalNode(exit_state,exit_message,index,exit_state_digit,Title:str):
    from bokeh.io import output_file, show
    from bokeh.models import ColumnDataSource
    from bokeh.plotting import figure
    from bokeh.io import output_notebook
    from bokeh.palettes import inferno

    output_file("ShowingCal.html")

    index = index
    exit_message = exit_message
    exit_state_string = exit_state
    exit_state_digit = exit_state_digit
      
    source = ColumnDataSource(data=dict(index=index, exit_state_digit=exit_state_digit,exit_message = exit_message,exit_state_string = exit_state_string, color=inferno(len(index))))
    
    TOOLTIPS = [
    ("Exit information", "@exit_message"),
    ("(x,y)", "($x, $y)"),
    ("Node index", "@index"),
    ("Node status", "@exit_state_string"),   
    ]

    p = figure( y_range=(0,1), x_range=(0,3000), plot_width=800, plot_height=800, title=Title, tooltips=TOOLTIPS)
    #print('step figure done')
    p.vbar(x="index", top="exit_state_digit", bottom=0, width=1, color='color',  source=source)
    #print('step hbar done')
    
    output_notebook()
    p.xgrid.grid_line_color = None
    #p.legend = False
    show(p)
    


In [16]:
from aiida.orm import CalcJobNode,WorkflowNode
from aiida.orm import QueryBuilder
import HelperPackage.DataProcessing.DataVisu as DV
import HelperPackage.Serialization.Serialization as SR

####### CalcNode 
qb = QueryBuilder()
qb.append(CalcJobNode)
CalcNode = qb.all()

serializer = SR.Serializer(CalcNode)
filepath = './output/CalcNode.json'
serializer.to_file(filepath,'ProcessNode')

######## WorkflowNode
qb = QueryBuilder()
qb.append(WorkflowNode)
WorkflowNodes = qb.all()

serializer = SR.Serializer(WorkflowNodes)
filepath2 = './output/WorkflowNode.json'
serializer.to_file(filepath2,'ProcessNode')

0

In [ ]:
#ShowCalNode(exit_state,exit_message,index,exit_state_digit)
### use scatter plot to replace this

In [17]:
calcArray = SR.deserialize_from_file(filepath,Node_type = 'ProcessNode')
WorkflowArray = SR.deserialize_from_file(filepath2,Node_type = 'ProcessNode')

Newdict1 = DV.GetWorkflowDict(WorkflowArray)
Newdict2 = DV.GetWorkflowDict(calcArray)

In [18]:
from aiida.orm import WorkflowNode


DV.ShowWorkflow(Newdict1,'Work Flow Node Information')
DV.ShowWorkflow(Newdict2,'Calculate Job Node Information')

Loading BokehJS ...

Loading BokehJS ...

# Data provenance health indicators:

In [ ]:

# SubtaskD1.i: Provenance
#Task: Database and provenance health: display the number of nodes who have no incomming and outgoing links, no incomming links (any number outgoing), and no outgoing links (any number incomming)

In [19]:
from aiida.orm import QueryBuilder
import HelperPackage.DataProcessing.DataVisu as DV
import HelperPackage.Serialization.Serialization as SR
import pandas as pd
import aiida
aiida.load_profile()

In [22]:
########## this cell will take some time
qb = QueryBuilder()
qb.append(Node)
Nodes = qb.all()

#### serialization to filepath
provenance_serializer = SR.Serializer(Nodes)
filepath = './output/provenance.json'
provenance_serializer.to_file(filepath,'Provenance')

The preprocessing took 125.56883120536804 seconds


0

In [21]:
#### deserialization from filepath
filepath = './output/provenance.json'
provenance = SR.deserialize_from_file(filepath,'Provenance')
provenance

,Node_Type,PK,FirstInput,FirstOutput
0,data.code.Code.,186,[],"[0f25075d-6a7f-48a1-b837-c419eafc3017, d50bb22..."
1,data.code.Code.,1,[],[]
2,data.code.Code.,2,[],[]
3,data.code.Code.,3,[],[]
4,data.code.Code.,4,[],[]
...,...,...,...,...
5809,data.structure.StructureData.,5809,[],[]
5810,data.structure.StructureData.,5810,[],[]
5811,data.structure.StructureData.,5811,[],[]
5812,data.structure.StructureData.,5813,[],[]


In [6]:
#### takes very long time
Mydict = DV.Count_In_Out(provenance)
print(Mydict)

{'No_Incoming': 5516, 'No_Outgoing': 5504, 'No_In&Out': 5432}


In [7]:
DV.Show_In_Out(Mydict)
### split and think about bar plot
# reduce complexity

Loading BokehJS ...